## Project 3

Creatied a several simple sqlite databases in python, created a table for inputting data, join those tables and select queries.

I completed a sql course using Azure Data Studio interface.  Our professor, Dr. Douglas Kline https://www.youtube.com/databasebydoug provided a DB server for the class to log in and access in order to complete the assignments. 

It seems easier to read sql queries using python and immediately begin analyzing the data.

In [1]:
import sqlite3
import pandas as pd

# create new database namedmyMusic
conn = sqlite3.connect('myMusic.db')
curs = conn.cursor()

# create new table (and check to make sure table does not exist with the same name)
# within myMusic named artist
curs.execute("""CREATE TABLE IF NOT EXISTS tbl_artist (
                artist_name text PRIMARY KEY,
                genre  text,
                song text);
            """)

#commit the changes 
conn.commit()


In [2]:
#Create a second table within the database named fanInfo
curs.execute("""CREATE TABLE IF NOT EXISTS tbl_info (
                artist_ID text PRIMARY KEY,
                dob  text,
                married text,
                albumSales int);
            """)
#commit the changes 
conn.commit()

In [3]:
#Check that it worked
check = '''
        SELECT name
        FROM sqlite_schema
        WHERE type = "table";
        '''
print(curs.execute(check).fetchall())

[('tbl_artist',), ('tbl_info',)]


In [4]:
# create list of artists to add to artist table; 
list_artist = [
                    ('Prince', 'Rock', 'Sign of the Times'),
                    ('Soundgarden', 'Alternative', 'My Wave'),
                    ('Public Enemy', 'Rap', 'Everything'),
                    ('Alice in Chains', 'Alternative', 'Heaven Beside You'),
                    ('Marvin Gaye', 'RB/Soul', 'Whats Going On'),
                    ('Aretha Franklin', 'RB/Soul', 'Aint No Way')
                ]


In [5]:
curs.executemany("INSERT INTO tbl_artist VALUES(?,?,?);", list_artist)
conn.commit()

In [6]:
artist_info = [
                    ('Prince', '06/07/1958', 'N', '15000000'),
                    ('Soundgarden', '07/20/1964', 'N', '14800000'),
                    ('Public Enemy', '08/01/1960', 'N', '4900000'),
                    ('Alice in Chains', '08/22/1967', 'N', '14000000'),
                    ('Marvin Gaye', '04/02/1939', 'N', '12300000'),
                    ('Aretha Franklin', '03/25/1942', 'N', '75000000')
                ]
    
    

In [7]:
conn.executemany("INSERT INTO tbl_info VALUES(?,?,?,?);", artist_info)

In [8]:
# I didn't quite this out.  the date of birth added to the db is for the lead singer. So, we will update 
# info table with the name of the lead singer and leave it blank if the artist sings solo.

curs.execute("""ALTER TABLE tbl_info ADD COLUMN lead
           """)


In [9]:
conn.execute("UPDATE tbl_info SET lead = 'Chris Cornell' WHERE artist_ID = 'Soundgarden'")
conn.execute("UPDATE tbl_info SET lead = 'Chuck D' WHERE artist_ID = 'Public Enemy'")
conn.execute("UPDATE tbl_info SET lead = 'Layne Staley' WHERE artist_ID = 'Alice in Chains'")

#commit the changes 
conn.commit()

In [10]:
# review the tables in myMusic
pd.read_sql("SELECT * FROM sqlite_schema WHERE type = 'table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,tbl_artist,tbl_artist,2,CREATE TABLE tbl_artist (\n art...
1,table,tbl_info,tbl_info,4,CREATE TABLE tbl_info (\n artis...


In [11]:
#display information in tbl_artist
pd.read_sql("SELECT artist_Name, genre, song FROM tbl_artist LIMIT 10;", conn)           

,artist_name,genre,song
0,Prince,Rock,Sign of the Times
1,Soundgarden,Alternative,My Wave
2,Public Enemy,Rap,Everything
3,Alice in Chains,Alternative,Heaven Beside You
4,Marvin Gaye,RB/Soul,Whats Going On
5,Aretha Franklin,RB/Soul,Aint No Way


In [12]:
# Join tbl artist and info and complete a simple query
# This format is similar to regular SQL
pd.read_sql("SELECT  artist_Name, genre, dob \
            FROM tbl_artist \
            LEFT JOIN tbl_info ON \
            tbl_artist.artist_Name = tbl_info.artist_ID \
            WHERE lead IS NOT NULL;", conn)

,artist_name,genre,dob
0,Soundgarden,Alternative,07/20/1964
1,Public Enemy,Rap,08/01/1960
2,Alice in Chains,Alternative,08/22/1967


In [13]:
pd.read_sql("SELECT  artist_Name, genre \
            FROM tbl_artist \
            LEFT JOIN tbl_info ON \
            tbl_artist.artist_Name = tbl_info.artist_ID \
            WHERE albumSales >15000000;", conn)

,artist_name,genre
0,Aretha Franklin,RB/Soul


In [14]:
#close db
conn.close()